# Imports and Constants

In [1]:
# File system
import os, os.path 
import pickle
from glob import glob
import sys

# Workflow
import random
import tqdm
import tqdm.notebook

# Computation
import numpy as np
import torch
import torch.nn as nn

# Data visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(rc={"figure.dpi":100, 'savefig.dpi':100})
sns.set_context('notebook')

torch.__version__

'1.8.0+cu111'

In [2]:
# Keys to the pickle objects
CITY = 'city'
LANE = 'lane'
LANE_NORM = 'lane_norm'
SCENE_IDX = 'scene_idx'
AGENT_ID = 'agent_id'
P_IN = 'p_in'
V_IN = 'v_in'
P_OUT = 'p_out'
V_OUT = 'v_out'
CAR_MASK = 'car_mask'
TRACK_ID = 'track_id'

# Encode Miami as 0, Pittsburgh as 1
MIA = 'MIA'
PIT = 'PIT'  
CITY_MAP = {MIA: 0, PIT: 1}

# Transformed data keys
LANE_IN = 'closest_lanes_in'
NORM_IN = 'closest_lane_norms_in'
LANE_OUT = 'closest_lanes_out'
NORM_OUT = 'closest_lane_norms_out'

In [3]:
# Dataset variables
VALIDATION_PATH = './val_data/'
ORIGINAL_PATH = './original_train_data/'
TRANSFORMED_TRAIN_PATH = './transformed_train_data'
TRANSFORMED_VAL_PATH = './transformed_val_data'

train_path = TRANSFORMED_TRAIN_PATH
val_path = TRANSFORMED_VAL_PATH

# Path to model predictions on test set
PREDICTION_PATH = './my_submission.csv'
# Header of predictions CSV file
CSV_HEADER = ['ID,'] + ['v' + str(i) + ',' for i in range(1, 60)] + ['v60', '\n']

# Get list of all training file names
original_files = glob(os.path.join(train_path, '*'))

# 80-20 train-test split
train_files = random.sample(original_files, int(len(original_files) * 0.8))
test_files = list(set(original_files) - set(train_files))

# Validation
val_files = glob(os.path.join(val_path, '*'))

In [14]:
NUM_AGENTS = 1
IN_LEN = 19
OUT_LEN = 30

# Controls how many features to use
N_FEAT_IN = 4
N_FEAT_OUT = 2

# Batch variables
BATCH_SIZE_TRAIN = 64
BATCH_SIZE_TEST = BATCH_SIZE_TRAIN
BATCH_SIZE_VAL = 32
N_WORKERS = 4
# Windows doesn't support anything but N_WORKERS = 0 for the DataLoader
if 'win' in sys.platform:
    N_WORKERS = 0
N_WORKERS

0

# Dataset Loading and Batching

In [33]:
class ArgoverseDataset(torch.utils.data.Dataset):
    def __init__(self, files):
        super(ArgoverseDataset, self).__init__()
        self.files = files
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):

        pkl_path = self.files[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        return data

In [35]:
def collate_train_test(batch):
    """ 
    Custom collate_fn function to be used for DataLoader.    
    The input tensor is organized as 19 rows, where each row has 4 columns: px, py, vx, vy
    
    The output tensor can be organized in 2 ways:
    A) The output tensor is organized as 120 rows, where each row has 1 column. 
    Each contiguous sequence of 4 elements is px, py, vx, vy
    
    B) The input tensor is organized as 30 rows, where each row has 4 columns: px, py, vx, vy
    """    
    inp = []     
    vel = []
    out = []
    agent_idxs = []
    scene_idxs = []
    for num, scene in enumerate(batch):
        # Get the target agent id
        agent_id = scene[AGENT_ID]        
        # Get the matrix of all agents
        track_id = scene[TRACK_ID]        
        # Get the location of the target agent in the matrix
        idx = np.nonzero(track_id[:, 0] == agent_id)[0][0]
        
        # Aliases of scene variables for convenience
        pin, pout, vin, vout = scene[P_IN], scene[P_OUT], scene[V_IN], scene[V_OUT]

        # Only include pos/vel
        inp_tens = np.concatenate((pin[idx], vin[idx]), axis=1)        
        out_tens = pout[idx] 
        
        inp.append(inp_tens)
        out.append(out_tens)  
        vel.append(vout[idx])

#         scene_idxs.append(scene[SCENE_IDX]) 
#         agent_idxs.append(idx)


    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    vel = torch.FloatTensor(vel)
    return [inp, out, scene_idxs, agent_idxs, vel]

In [36]:
def collate_val(batch):
    """ 
    Custom collate_fn for validation dataset. The validation data do not contain output values.   
    The input tensor is organized as 19 rows, where each row has 4 columns: px, py, vx, vy
    """   
    inp = []
    scene_idxs = []
    agent_idxs = []
    
    for num, scene in enumerate(batch):
        # Get the target agent id
        agent_id = scene[AGENT_ID]        
        # Get the matrix of all agents
        track_id = scene[TRACK_ID]        
        # Get the location of the target agent in the matrix
        idx = np.nonzero(track_id[:, 0] == agent_id)[0][0]
                
        # Aliases of scene variables for convenience
        pin, vin = scene[P_IN], scene[V_IN]       
        inp_tens = np.concatenate((pin[idx], vin[idx]), axis=1)
        
        inp.append(inp_tens)        
        scene_idxs.append(scene[SCENE_IDX])
        agent_idxs.append(idx)
        
    inp = torch.FloatTensor(inp)    
    return [inp, scene_idxs, agent_idxs]

In [37]:
# Initiliaze datasets and loaders
train_dataset = ArgoverseDataset(train_files)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, 
                                           shuffle=True, collate_fn=collate_train_test, 
                                           num_workers=N_WORKERS)
test_dataset = ArgoverseDataset(test_files)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE_TEST, 
                                          shuffle=False, collate_fn=collate_train_test, 
                                          num_workers=N_WORKERS)
val_dataset = ArgoverseDataset(val_files)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, 
                                         shuffle=False, collate_fn=collate_val,
                                         num_workers=0)

In [38]:
# # Look at one data sample      
for _, (data, target, agent_idxs, scene_idxs, vel) in enumerate(train_loader):
    print(data.shape)
    print(target.shape) 
    break

torch.Size([64, 19, 4])
torch.Size([64, 30, 2])


In [39]:
# # Look at one data sample      
for _, (data, agent_idxs, scene_idxs) in enumerate(val_loader):
    print(data.shape)
    break

torch.Size([32, 19, 4])


# Training Workflow

In [40]:
def train(model, device, train_loader, optimizer, epoch, velmodel, velopt):
    # Set the model into training mode
    model.train()    
    velmodel.train()
    
    # Define the loss function.
    criterion = torch.nn.MSELoss(reduction='mean')
    velcrit = torch.nn.MSELoss(reduction='mean')

    total_loss = 0
    vel_total_loss = 0
    
    for i in range(epoch):
        iterator = tqdm.notebook.tqdm(train_loader, total=int(len(train_loader)))
        for _, batch in enumerate(iterator):
            data, target, _, _, vel = batch
            data, target = data.to(device), target.to(device)
            vel = vel.to(device)

            optimizer.zero_grad()  
            velopt.zero_grad()
            
            outvel = velmodel(data)  
            out = model(data, vel)
            # Compute the loss       
            velloss = torch.sqrt(velcrit(outvel, vel))
            loss = torch.sqrt(criterion(out, target)) 
            vel_total_loss += velloss.item()
            total_loss += loss.item()

            # Perform backpropagation
            velloss.backward()
            loss.backward()
            # Update the weights
            velopt.step()
            optimizer.step()            

            # Update the progress bar for tqdm
            iterator.set_postfix(train_loss_vel=velloss.item(), train_loss=loss.item())
            
    return (total_loss * BATCH_SIZE_TRAIN) / len(train_files), (vel_total_loss * BATCH_SIZE_TRAIN) / len(train_files)

In [41]:
def test(model, device, test_loader, velmodel):
    model.eval()    
    velmodel.eval()
    criterion = torch.nn.MSELoss(reduction='mean') 
    velcrit = torch.nn.MSELoss(reduction='mean')

    iterator = tqdm.notebook.tqdm(test_loader, total=int(len(test_loader)))
    total_loss = 0
    vel_total_loss = 0
    
    for _, batch in enumerate(iterator):
        data, target, _, _, vel = batch
        data, target = data.to(device), target.to(device)
        vel = vel.to(device)
        
        with torch.no_grad():
            outvel = velmodel(data)
            out = model(data, outvel)            
            velloss = torch.sqrt(criterion(outvel, vel))   
            loss = torch.sqrt(criterion(out, target))        
            total_loss += loss.item()
            vel_total_loss += velloss.item()

            iterator.set_postfix(test_loss_vel=velloss.item(), test_loss=loss.item())
        
    return (total_loss * BATCH_SIZE_TEST) / len(test_files), (vel_total_loss * BATCH_SIZE_TEST) / len(test_files)

In [42]:
def train_test(model, DEVICE, train_loader, test_loader, optimizer, NUM_EPOCH, velmodel, velopt):
    for t in range(1, NUM_EPOCH + 1):
        train_loss = train(model, DEVICE, train_loader, optimizer, 1, velmodel, velopt)
        test_loss = test(model, DEVICE, test_loader, velmodel)
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        print(f'Epoch {t}: train_loss = {train_loss}, test_loss = {test_loss}')         

In [73]:
def validate(model, device, val_loader, path, velmodel):
    """
    path: path to csv file to write predictions
    """
    model.eval() 
    velmodel.eval()
    
    # Prep the output file
    with open(PREDICTION_PATH, "w") as csv_file:
        # Clear the csv file before appending data to it
        csv_file.truncate()
        # Write the header to the csv file
        csv_file.writelines(CSV_HEADER)    
    
    # Make predictions
    with open(path, "a") as pred_file:        
        iterator = tqdm.notebook.tqdm(val_loader, total=int(len(val_loader)))
        
        for _, batch in enumerate(iterator):
            data, scene_idxs, _ = batch
            data = data.to(device) 
            
            with torch.no_grad():
                outvel = velmodel(data)
                output = model(data, outvel)
                # Convert the Tensor from GPU -> CPU -> NumPy array
                np_out = output.cpu().detach().numpy()
                
                # Store only the predictions for the target agent and keep the positions, not the velocities
                batch_size = np_out.shape[0]
                
                pred = np.zeros((batch_size, 60))
                # The output should be a (batch size, time steps, num features out) tensor 
                # where the first two features are the out position x, y
                for i in range(batch_size):
                    pred[i] = np_out[i, :, :2].flatten()                          

                # Form comma-separated string
                s = []
                for i in range(pred.shape[0]):
                    s.append(','.join([str(scene_idxs[i])] + [str(v) for v in pred[i]]) + '\n')

                # Write data to file
                pred_file.writelines(s)

# Model Initialization

In [44]:
# class ArgoNet(torch.nn.Module):
#     """
#     Neural Network class - linear regression
#     """
#     def __init__(self, device):
#         super(ArgoNet, self).__init__() 
        
#         self.device = device        
#         # Linear regression for 1 agent     
#         self.fc = nn.Linear((IN_LEN * N_FEAT_IN) + 60, OUT_LEN * N_FEAT_OUT)   
    
#     def forward(self, x, vel):
#         x = x.reshape(x.shape[0], -1)
#         vel = vel.reshape(vel.shape[0], -1)
#         z = torch.cat((x, vel), dim=1)
#         z = self.fc(z)            
#         z = z.view(z.size(0), OUT_LEN, N_FEAT_OUT)  
#         return z

In [145]:
class ArgoNet(torch.nn.Module):
    def __init__(self, device):
        super(ArgoNet, self).__init__() 
        
        self.device = device        
        self.convA = nn.Sequential(
            nn.Conv1d(19, 12, 3),
            nn.SELU()
        )
        self.convB = nn.Sequential(
            nn.Conv1d(30, 18, 1),
            nn.SELU()
        )   
        self.fc = nn.Linear(60, 60)   
    
    def forward(self, x, vel):
        x = self.convA(x)
        vel = self.convB(vel)
        z = torch.cat((x, vel), dim=1)
        z = z.view(z.shape[0], -1)
        z = self.fc(z)
        z = z.view(z.shape[0], 30, 2)
        return z

In [146]:
# class VelNet(torch.nn.Module):
#     """
#     Predicts output velocities given input positions and velocities
#     """
#     def __init__(self, device):
#         super(VelNet, self).__init__()         
#         self.device = device        
#         self.fc = nn.Linear(19 * 4, 30 * 2)
   
    
#     def forward(self, x):
#         x = x.view(x.shape[0], -1)
#         x = self.fc(x)
#         x = x.view(x.shape[0], 30, 2)
#         return x

In [161]:
class VelNet(torch.nn.Module):
    def __init__(self, device):
        super(VelNet, self).__init__()         
        self.device = device    
        self.conv = nn.Conv1d(19, 12, 1)
        self.act = nn.SELU()
        self.fc = nn.Sequential(
            nn.Linear(12 * 4, 120),
            nn.SELU(),
            nn.Linear(120, 60)
        )
   
    
    def forward(self, x):
        x = self.conv(x)
        x = self.act(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        x = x.view(x.shape[0], 30, 2)
        return x

In [162]:
# Code to save and reload a model
MODEL_STATE = 'model_state_dict'
OPTIMIZER_STATE = 'optimizer_state_dict'
EPOCH_STATE = 'epoch'
LOSS_STATE = 'loss'
BATCH_STATE = 'batch'

def save_model(path, model_state_dict, optimizer_state_dict, epoch, loss, batch):
    to_save = {
        MODEL_STATE: model_state_dict,
        OPTIMIZER_STATE: optimizer_state_dict,
        EPOCH_STATE: epoch,
        LOSS_STATE: loss,
        BATCH_STATE: batch
    }
    torch.save(to_save, path)
    
def load_model(path, model_to_load, optimizer_to_load):
    checkpoint = torch.load(path)
    model_to_load.load_state_dict(checkpoint[MODEL_STATE])
    optimizer_to_load.load_state_dict(checkpoint[OPTIMIZER_STATE])
    return checkpoint[EPOCH_STATE], checkpoint[LOSS_STATE], checkpoint[BATCH_STATE]

In [163]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ArgoNet(DEVICE).to(DEVICE)
velmodel = VelNet(DEVICE).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters())
velopt = torch.optim.Adam(velmodel.parameters())


print(f"Number of Vel model parameters is {sum(p.numel() for p in velmodel.parameters())}")
print(f"Number of Argo model parameters is {sum(p.numel() for p in model.parameters())}")
NUM_EPOCH = 1

# used for visualizing the loss
train_losses = []
test_losses = []

Number of Vel model parameters is 13380
Number of Argo model parameters is 4914


In [164]:
# Use these two lines to save a model
# save_model('sample.tar', model.state_dict(), optimizer.state_dict(), NUM_EPOCH, 
#            (train_losses[-1], test_losses[-1]), BATCH_SIZE_TRAIN)

In [165]:
# Reload a model
# model = ArgoNet(DEVICE).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters())
# load_model('one_hid_one_agent.tar', model, optimizer)
# train_losses.clear()
# test_losses.clear()
# num_params = sum(p.numel() for p in model.parameters())   
# print(f"Number of model parameters is {num_params}")

# Evaluation

In [166]:
train_test(model, DEVICE, train_loader, test_loader, optimizer, NUM_EPOCH, velmodel, velopt)

KeyboardInterrupt: 

In [119]:
validate(model, DEVICE, val_loader, PREDICTION_PATH, velmodel)

In [ ]:
# def main():
#     # Initiliaze datasets and loaders
#     train_dataset = ArgoverseDataset(train_files)
#     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, 
#                                                shuffle=True, collate_fn=collate_train_test, 
#                                                num_workers=N_WORKERS)
#     test_dataset = ArgoverseDataset(test_files)
#     test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE_TEST, 
#                                               shuffle=False, collate_fn=collate_train_test, 
#                                               num_workers=N_WORKERS)
#     for t in range(1, NUM_EPOCH + 1):
#         # TRAIN
#         model.train()    
#         velmodel.train()
#         criterion = torch.nn.MSELoss(reduction='mean')
#         velcrit = torch.nn.MSELoss(reduction='mean')
#         total_loss = 0
#         vel_total_loss = 0

#         iterator = tqdm.notebook.tqdm(train_loader, total=int(len(train_loader)))
#         for _, batch in enumerate(iterator):
#             data, target, _, _, vel = batch
#             data, target = data.to(device), target.to(device)
#             vel = vel.to(device)

#             optimizer.zero_grad()  
#             velopt.zero_grad()

#             outvel = velmodel(data)  
#             out = model(data, outvel.data)
#             # Compute the loss       
#             velloss = torch.sqrt(velcrit(outvel, vel))
#             loss = torch.sqrt(criterion(out, target)) 
#             vel_total_loss += velloss.item()
#             total_loss += loss.item()

#             # Perform backpropagation
#             velloss.backward()
#             loss.backward()
#             # Update the weights
#             velopt.step()
#             optimizer.step()            

#             # Update the progress bar for tqdm
#             iterator.set_postfix(train_loss_vel=velloss.item(), train_loss=loss.item()) 
            
#         total_loss = (total_loss * BATCH_SIZE_TRAIN) / len(train_files)
#         vel_total_loss = (vel_total_loss * BATCH_SIZE_TRAIN) / len(train_files)
#         train_losses.append((total_loss, vel_total_loss))
   
#         # TEST
#         model.eval()    
#         velmodel.eval()
#         criterion = torch.nn.MSELoss(reduction='mean') 
#         velcrit = torch.nn.MSELoss(reduction='mean')

#         iterator = tqdm.notebook.tqdm(test_loader, total=int(len(test_loader)))
#         total_loss = 0
#         vel_total_loss = 0
    
#         for _, batch in enumerate(iterator):
#             data, target, _, _, vel = batch
#             data, target = data.to(device), target.to(device)
#             vel = vel.to(device)

#             with torch.no_grad():
#                 outvel = velmodel(data)
#                 out = model(data, outvel)            
#                 velloss = torch.sqrt(criterion(outvel, vel))   
#                 loss = torch.sqrt(criterion(out, target))        
#                 total_loss += loss.item()
#                 vel_total_loss += velloss.item()

#                 iterator.set_postfix(test_loss_vel=velloss.item(), test_loss=loss.item())

#         total_loss = (total_loss * BATCH_SIZE_TEST) / len(test_files)
#         vel_total_loss = (vel_total_loss * BATCH_SIZE_TEST) / len(test_files)        
#         test_losses.append((total_loss, vel_total_loss))

#         print(f'Epoch {t}: train_loss = {train_losses[-1]}, test_loss = {test_losses[-1]}')         

In [ ]:
# if __name__ == '__main__':
#     main()

In [ ]:
# train_test(model, DEVICE, train_loader, test_loader, optimizer, NUM_EPOCH, velmodel, velopt)
# validate(model, DEVICE, val_loader, PREDICTION_PATH, velmodel)

# Loss Visualization

In [ ]:
def visualize_loss(losses):
    """
    Plots the losses over each training iteration. 
    Assumes that each element of the 'losses' list corresponds to the loss after each batch of train()
    """
    t_iter = np.arange(1, len(losses) + 1, 1, dtype=int)
    ax = sns.scatterplot(x=t_iter, y=losses, alpha=0.5)    
    ax.set_xlabel('Batch iteration number')
    ax.set_ylabel('Root-mean-square loss')
    ax.set_title('Batch Iteration vs. Root-Mean-Square Loss')
    plt.savefig('lossViter')

In [ ]:
visualize_loss(train_losses)

# Ground Truth Comparison

In [ ]:
def visualize_predictions(model, device, loader):
    """
    Compares some randomly selected data samples to the model's predictions
    """
    model.eval()
    
    # Get a batch of data
    _, (inp, out, scene_idxs, agent_idxs, masks) = next(enumerate(loader))
    
    # Move tensors to chosen device
    inp, out = inp.to(device), out.to(device)
    
    # Sample number
    i = 0
    
    # Scene idx
    scene_idx = scene_idxs[i]
        
    # Get contiguous arrays of the ground truth output positions
    truth = target[i].cpu().detach().numpy()
    x = truth[:, 0]
    y = truth[:, 1] 
        
    # Get contiguous arrays of the prediction output positions
    output = model(inp)    
    pred = output[i].cpu().detach().numpy()
    xh = pred[:, 0]
    yh = pred[:, 0]    
    
    # Plot the ground truth and prediction positions
    fig, (ax) = plt.subplots(nrows=1, ncols=1, figsize=(3, 3))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Scene ' + str(scene_idx))
    ax.scatter(x, y, label='Ground Truth')
    ax.scatter(xh, yh, label='Prediction')
    ax.legend()

In [ ]:
visualize_predictions(model, DEVICE, train_loader)